# Check GPU version.

In [ ]:
!nvidia-smi

# Mount google drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Install TensorFlow-1.14 GPU.

In [ ]:
# Select TensorFlow-1.x version.
%tensorflow_version 1.x

# Uninstall previous TensorFlow version.
!pip uninstall tensorflow -y 1>/dev/null 2>/dev/null 
!pip uninstall tensorflow-gpu -y 1>/dev/null 2>/dev/null 

# Install TensorFlow-1.14 and Keras-2.2.4.
!pip install --upgrade tensorflow==1.14.0 1>/dev/null 2>/dev/null 
!pip install --upgrade tensorflow-gpu==1.14.0 1>/dev/null 2>/dev/null 
!pip install --upgrade keras==2.2.4 1>/dev/null 2>/dev/null

# Restart the runtime.

# Set the root directory.

In [ ]:
import os

root_dir = '/content/'
os.chdir(root_dir)

!ls -al

# Import TensorFlow-1.14.

In [ ]:
try:
  %tensorflow_version 1.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras import backend as K

import tensorflow.keras.layers as layers
import tensorflow.keras.models as models

import numpy as np
np.random.seed(7)

import matplotlib.pyplot as plot

print(tf.__version__)

# Create feature extractor.

In [ ]:
image_shape = (224, 224, 3)
number_of_features = 256

In [ ]:
from tensorflow.compat.v1.keras.applications import ResNet50

In [ ]:
base_model = ResNet50(include_top=False, weights=None, input_shape=image_shape, pooling='avg')

In [ ]:
image_features = base_model.output
image_features = layers.Flatten(name='flatten')(image_features)
image_features = layers.Dense(number_of_features, name='image_features')(image_features)
image_features = layers.Lambda(lambda  x: K.l2_normalize(x, axis=1))(image_features)

In [ ]:
resnet50_features = models.Model(inputs=base_model.input, outputs=image_features, name='model')

In [ ]:
import os

def model_filename():
  return('model.h5')

def weight_root_dir():
  return('/content/drive/My Drive/')

def model_gdrive_filename():    
  return(os.path.join(weight_root_dir(), model_filename()))

In [ ]:
resnet50_features.load_weights(model_gdrive_filename())

# Normalize input image.

In [ ]:
import cv2

def normalize_image(image_filename): 
  input_image = cv2.imread(image_filename) 
  input_image = cv2.resize(input_image, (image_shape[0], image_shape[1])) 
  input_image = cv2.cvtColor(input_image, cv2.COLOR_BGR2RGB)
  input_image = input_image / 255.0  
  input_image = np.expand_dims(input_image, axis=0)   
  return( input_image )

# Compute image features.

In [ ]:
def compute_image_features(image_filename): 
  current_image = normalize_image(image_filename)
  current_features = resnet50_features.predict(current_image)
  current_features = current_features[0]  
  return( current_features )

# Register persons using single image.

In [ ]:
def compute_features(aligned_image_dir):
  image_features = {}
  image_filenames = os.listdir(aligned_image_dir)
  for image_filename in image_filenames:
    identifier = image_filename.split('.jpg')
    identifier = identifier[0]

    image_path = os.path.join(aligned_image_dir, image_filename)
    current_features = compute_image_features(image_path)

    image_features[identifier] = current_features
  return(image_features)

In [ ]:
aligned_image_dir = '/content/drive/My Drive/aligned_images/'
test_image_dir = '/content/drive/My Drive/test_images/'

In [ ]:
image_features = compute_features(aligned_image_dir)

# Identify person using pre-computed image features.

In [ ]:
def identify_person(image_features, current_features, maximum_distance=0.7):
  person_name = 'unknown'
  minimum_distance = float('inf')

  for person in image_features:
    person_features = image_features[person]
    current_distance = np.linalg.norm(person_features - current_features)

    if(current_distance < minimum_distance):
      minimum_distance = current_distance
      person_name = person

  if(minimum_distance > maximum_distance):
    person_name = 'unknown'

  return(person_name, minimum_distance)

In [ ]:
def identify_person(image_features, current_features, minimum_similarity=0.7):
  person_name = 'unknown'
  maximum_similarity = 0.

  for person in image_features:
    person_features = image_features[person]
    current_similarity = np.dot(person_features, np.transpose(current_features))

    if(current_similarity > maximum_similarity):
      maximum_similarity = current_similarity
      person_name = person

  if(maximum_similarity < minimum_similarity):
    person_name = 'unknown'

  return(person_name, maximum_similarity)

# Test one-shot recognition.

In [ ]:
def identify_persons(image_features, test_image_dir):
  image_filenames = os.listdir(test_image_dir)
  for image_filename in image_filenames:

    identifier = image_filename.split('.jpg')
    identifier = identifier[0]

    image_path = os.path.join(test_image_dir, image_filename)
    current_features = compute_image_features(image_path)
    person_name, minimum_distance = identify_person(image_features, current_features)
    print('**************************************************')
    print('ground truth -', identifier)
    print('predicted -',person_name)
    print('distance -',minimum_distance)
    print('**************************************************')

In [ ]:
identify_persons(image_features, test_image_dir)